In [2]:
import keras

base_model = keras.applications.EfficientNetB0(include_top=False) 

In [3]:
from keras.utils import image_dataset_from_directory

In [5]:
train_ds, val_ds = image_dataset_from_directory(
    "C:/images", image_size=(224, 224),
    validation_split=0.2, subset='both', seed=42
)

Found 2148 files belonging to 2 classes.
Using 1719 files for training.
Using 429 files for validation.


In [6]:
base_model.trainable=False # 기존 학습된 커널의 가중치를 변경하지 않음 

In [8]:
inputs = keras.layers.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(128, activation='relu')(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)

In [ ]:
adam = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-model.keras', save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

hist = model.fit(train_ds, epochs=20, validation_data=val_ds, callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 60s 966ms/step - accuracy: 0.8384 - loss: 0.3583 - val_accuracy: 0.9697 - val_loss: 0.1000
Epoch 2/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 49s 905ms/step - accuracy: 0.9681 - loss: 0.1005 - val_accuracy: 0.9744 - val_loss: 0.0771
Epoch 3/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 47s 881ms/step - accuracy: 0.9769 - loss: 0.0652 - val_accuracy: 0.9744 - val_loss: 0.0835
Epoch 4/20
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 739ms/step - accuracy: 0.9721 - loss: 0.0647